<div id="TOP" />
<a href="#END" > END of Notebook (filter) </a>

In [1]:
# -- CONFIG ------------------------------------
LAB_TAG=${PWD##*/} 
LAB_NAME="TFI_Azure_Lab9. Scaling and loadbalancing web servers"
LAB_WEIGHT=90
LAB_MODE="Terraform"
# ----------------------------------------------

# NB=""; cd ~/nb/2021-Terraform/content/tf-labs/XXXX; set +x
DIE() { echo "[$PWD] Error: $*"; banner "ERROR"; exit 1; }
NBTOOL_RC=~/scripts/nbtool.rc; [ ! -f $NBTOOL_RC ] && DIE "No such script as $NBTOOL_RC"
echo "==== Reading nbtool functions from '$NBTOOL_RC' ===="; . $NBTOOL_RC
EXCL_FN_INIT_NOTEBOOK
## --INCLUDE--SECTION--
## --EXCLUDE--SECTION--

==== Reading nbtool functions from '/home/student/scripts/nbtool.rc' ====
NB_DIR=/home/student/nb/2022-Terraform/content/tf-labs/9.ScalingAndLoadBalancingWebservers

==== Setting TF_DATA_DIR ====
TF_DATA_DIR=/home/student/dot.terraform

==== Setting AWS vars for student20 account ====
==== Terraform Lab weight[90] / tag[9.ScalingAndLoadBalancingWebservers] / name[Lab 9. Scaling and loadbalancing web servers] ====
NB_DIR=/home/student/nb/2022-Terraform/content/tf-labs/9.ScalingAndLoadBalancingWebservers
NB=/home/student/nb/2022-Terraform/content/tf-labs/9.ScalingAndLoadBalancingWebservers/README.ipynb

==== Filtering current notebook [/home/student/nb/2022-Terraform/content/tf-labs/9.ScalingAndLoadBalancingWebservers/README.ipynb] ====
-- . ~/scripts/nbtool.rc

-- /usr/bin/python3 ~/scripts/nbtool.py -f ~/nb/2022-Terraform/content/tf-labs/9.ScalingAndLoadBalancingWebservers/README.ipynb
---- no VARS_SEEN ----
i/p nb has 0 variables
/home/student/nb/2022-Terraform/content/tf-labs/9.Scali

#EXCLUDE
---
title: 'Lab 9. Scaling and loadbalancing web servers'
date: 2019-02-11T19:27:37+10:00
weight: 90
---

# --INCLUDE--SECTION--

## Background:

In this lab, we will take the web servers that we created in lab 8 and deploy them to an Amazon AWS Auto Scaling Group and also deploy a load balancer to share the traffic between them.

Tasks.
- As before, create a new directory lab 9, and create three files, resources.tf, main.tf and vars.tf.
- You will create an auto scaling group with a desired capacity of three servers.
- Use the template files from the previous lab.

## Tasks

### 1. Create a new Lab directory ~/labs/lab9



In [2]:
mkdir -p ~/labs/lab9
cd        ~/labs/lab9

In [3]:
cp -a ~/labs/lab8a/secgroup.tf .
cp -a ~/labs/lab8a/data_ami.tf .

### 2. Create an auto-scaling group and a launch template for the web servers

#### Lab 8 Variables:

In lab 8 we defined the following variables:

*  "aws_region"

*  "names"

*  "key_name"

*  "http_port"

*  "server_text"



#### Lab 9 Variables:

In lab 9 we will define the following variables:

*  "aws_region"

*  "name"   (*name* not *names*)

*  "key_name"

*  "web_server_http_port"

*  "alb_http_port"

*  "server_text"

Note that we no longer define individual "*names*" for the web servers, but let the Autoscaling Group create names for us.

So we define a "*name*" variable to be used by the Autoscaling group.

We also change the name of "*http_port*" to *web_server_http_port* - this is an internal port number used by the web servers.

We now define *alb_http_port* as the port defined for the Application Load Balancer.  External requests will be made to this port.


#### Lab 8 Outputs:

In Lab 8 we output the public_ips and the URLs of each individual web server

#### Lab 9 Outputs:

Now in Lab 9 we will use a front end ALB, so we output it's dns_name and also it's url.


#### Lab 8 Resouces:
**Note**: In lab 8 we created the following resources in the root module:

* resource "aws_instance" "web_server" - https://registry.terraform.io/providers/hashicorp/aws/latest/docs/resources/instance

* resource "aws_security_group" "web_server" - https://registry.terraform.io/providers/hashicorp/aws/latest/docs/resources/security_group

* resource "aws_security_group_rule" "allow_http_inbound" - https://registry.terraform.io/providers/hashicorp/aws/latest/docs/resources/security_group_rule

* resource "aws_security_group_rule" "allow_ssh_inbound"

* resource "aws_security_group_rule" "allow_all_outbound"


#### Lab 9 Resouces:

To support rolling deployments we will set the create_before_destroy parameter in the ASG livecycle to true.

In implementing this lab we will not directly define the individual web servers, we will define an *AutoscalingGroup* which will have a *launch_configuration* to create the web server instances for us 

* resource "aws_autoscaling_group" "web_servers" - https://registry.terraform.io/providers/hashicorp/aws/latest/docs/resources/autoscaling_group


<img src="ASG.PNG" />

**Note**: the lifecycle structure shown bottom left of the above schema is to be added in all ASG/ALB related resources - see comment in the code below.

For the auto scaling group we will define a minimum and maximum of three servers.

### 3. create the autoscaling group resource definition

Add the following definition to the main.tf resources to define the AutoScaling Group:

In [4]:
#EXCLUDE

cat > main.tf <<EOF
resource "aws_autoscaling_group" "web_servers" {
  # Note that we intentionally depend on the Launch Configuration name so that creating a new Launch Configuration
  # (e.g. to deploy a new AMI) creates a new Auto Scaling Group. This will allow for rolling deployments.
  name = aws_launch_configuration.web_servers.name

  launch_configuration = aws_launch_configuration.web_servers.name

  min_size         = 3
  max_size         = 3
  desired_capacity = 3
  min_elb_capacity = 3

  # Deploy all the subnets (and therefore AZs) available
  vpc_zone_identifier = data.aws_subnets.default.ids

  # Automatically register this ASG's Instances in the ALB and use the ALB's health check to determine when an Instance
  # needs to be replaced
  health_check_type = "ELB"

  target_group_arns = [aws_alb_target_group.web_servers.arn]

  tag {
    key                 = "Name"
    value               = var.name
    propagate_at_launch = true
  }
  # To support rolling deployments, we tell Terraform to create a new ASG before deleting the old one. Note: as
  # soon as you set create_before_destroy = true in one resource, you must also set it in every resource that it
  # depends on, or you'll get an error about cyclic dependencies (especially when removing resources).
  lifecycle {
    create_before_destroy = true
  }

  # This needs to be here to ensure the ALB has at least one listener rule before the ASG is created. Otherwise, on the
  # very first deployment, the ALB won't bother doing any health checks, which means min_elb_capacity will not be
  # achieved, and the whole deployment will fail.
  depends_on = [aws_alb_listener.http]
}

EOF

cat > vars.tf <<EOF

variable aws_region {
    default = "us-west-1"
}

variable name {
    type = string
    default = "alb-webserver"
}

# CHANGED FROM http_port:
variable web_server_http_port {
    type = string
    default = "8080"
}

variable server_text {
    type = string
    default = "Your server is up !!"
}

variable key_name {
    default = "asg-webservers-key"
}

EOF

cat > keypair.tf <<EOF
# Create a TLS private key  resource
resource "tls_private_key" "mykey" {
  algorithm = "RSA"
  rsa_bits  = 4096
}

# Create an AWS 'key pair' resource usable for ssh from the TLS private key
resource "aws_key_pair" "generated_key" {
  key_name   = var.key_name
  public_key = tls_private_key.mykey.public_key_openssh
}


output "ssh_rsa_pub_key" { value = tls_private_key.mykey.public_key_openssh }

output "ssh_pem_key" {
  value     = tls_private_key.mykey.private_key_pem
  sensitive = true
}

resource "local_file" "pem_key" {
  filename        = "key.pem"
  content         = tls_private_key.mykey.private_key_pem
  file_permission = 0600
}

EOF


In [5]:
cat keypair.tf

# Create a TLS private key  resource
resource "tls_private_key" "mykey" {
  algorithm = "RSA"
  rsa_bits  = 4096
}

# Create an AWS 'key pair' resource usable for ssh from the TLS private key
resource "aws_key_pair" "generated_key" {
  key_name   = var.key_name
  public_key = tls_private_key.mykey.public_key_openssh
}


output "ssh_rsa_pub_key" { value = tls_private_key.mykey.public_key_openssh }

output "ssh_pem_key" {
  value     = tls_private_key.mykey.private_key_pem
  sensitive = true
}

resource "local_file" "pem_key" {
  filename        = "key.pem"
  content         = tls_private_key.mykey.private_key_pem
  file_permission = 0600
}



In [6]:
cat main.tf

resource "aws_autoscaling_group" "web_servers" {
  # Note that we intentionally depend on the Launch Configuration name so that creating a new Launch Configuration
  # (e.g. to deploy a new AMI) creates a new Auto Scaling Group. This will allow for rolling deployments.
  name = aws_launch_configuration.web_servers.name

  launch_configuration = aws_launch_configuration.web_servers.name

  min_size         = 3
  max_size         = 3
  desired_capacity = 3
  min_elb_capacity = 3

  # Deploy all the subnets (and therefore AZs) available
  vpc_zone_identifier = data.aws_subnets.default.ids

  # Automatically register this ASG's Instances in the ALB and use the ALB's health check to determine when an Instance
  # needs to be replaced
  health_check_type = "ELB"

  target_group_arns = [aws_alb_target_group.web_servers.arn]

  tag {
    key                 = "Name"
    value               = var.name
    propagate_at_launch = true
  }
  # To support rolling deployments, we tell Terraform to cr

**Note**: the use of depends_on above to ensure that the ASG group doesn't get created before the load balancer listening service is created.

* resource "aws_launch_configuration" "web_servers" - https://registry.terraform.io/providers/hashicorp/aws/latest/docs/data-sources/launch_configuration

Add the following definition to the main.tf resources to define the LaunchConfiguration used by the AutoScaling Group:



In [7]:
#EXCLUDE

cat > main.tf.2 <<R_EOF

resource "aws_launch_configuration" "web_servers" {
  image_id      = data.aws_ami.ubuntu.id
  instance_type = "t2.micro"

  key_name        = var.key_name
  security_groups = [aws_security_group.web_server.id]

  # To keep this example simple, we run a web server as a User Data script. In real-world usage, you would typically
  # install the web server and its dependencies in the AMI.
  user_data = <<-EOF
              #!/bin/bash
              echo "\${var.server_text}" > index.html
              nohup busybox httpd -f -p "\${var.web_server_http_port}" &
              EOF

  # When used with an aws_autoscaling_group resource, the aws_launch_configuration must set create_before_destroy to
  # true. Note: as soon as you set create_before_destroy = true in one resource, you must also set it in every resource
  # that it depends on, or you'll get an error about cyclic dependencies (especially when removing resources).
  lifecycle {
    create_before_destroy = true
  }
}

R_EOF

In [8]:
#EXCLUDE

cat main.tf.2 >> main.tf


In [9]:
EXEC cat main.tf.2
NO_EXEC tail main.tf


resource "aws_launch_configuration" "web_servers" {
  image_id      = data.aws_ami.ubuntu.id
  instance_type = "t2.micro"

  key_name        = var.key_name
  security_groups = [aws_security_group.web_server.id]

  # To keep this example simple, we run a web server as a User Data script. In real-world usage, you would typically
  # install the web server and its dependencies in the AMI.
  user_data = <<-EOF
              #!/bin/bash
              echo "${var.server_text}" > index.html
              nohup busybox httpd -f -p "${var.web_server_http_port}" &
              EOF

  # When used with an aws_autoscaling_group resource, the aws_launch_configuration must set create_before_destroy to
  # true. Note: as soon as you set create_before_destroy = true in one resource, you must also set it in every resource
  # that it depends on, or you'll get an error about cyclic dependencies (especially when removing resources).
  lifecycle {
    create_before_destroy = true
  }
}



In [10]:
cat vars.tf


variable aws_region {
    default = "us-west-1"
}

variable name {
    type = string
    default = "alb-webserver"
}

# CHANGED FROM http_port:
variable web_server_http_port {
    type = string
    default = "8080"
}

variable server_text {
    type = string
    default = "Your server is up !!"
}

variable key_name {
    default = "asg-webservers-key"
}



### 4. Show the hostname

Modify the above so that the served up text includes the serving hostname.

Add "from $(hostname)" into the index.html,

i.e. change the line:
```echo "${var.server_text}"```
to be
```echo "from $(hostname): ${var.server_text}"```

In [11]:
#EXCLUDE
pwd
#sed -ie.bak
#sed -e      's/\${var.server_text}/from \$(hostname): \${var.server_text}/' main.tf | grep server_text
sed  -i.bak -e's/\${var.server_text}/from \$(hostname): \${var.server_text}/' main.tf

/home/student/labs/lab9


### 5. Modify the security group

The *web_server* security group will be the same, but using  the new *web_server_http_port* variable.

* resource "aws_security_group" "web_server"

* resource "aws_security_group_rule" "web_server_allow_http_inbound"

* resource "aws_security_group_rule" "web_server_allow_ssh_inbound"

* resource "aws_security_group_rule" "web_server_allow_all_outbound"

In [12]:
#EXCLUDE

cat >secgroup.tf <<EOF

resource "aws_security_group" "web_server" {
  name_prefix = "tfi-lab9"
}

resource "aws_security_group_rule" "web_server_allow_http_inbound" {
  type              = "ingress"
  
  # CHANGED to web_server_http_port:
  from_port         = var.web_server_http_port
  to_port           = var.web_server_http_port
  
  protocol          = "tcp"
  security_group_id = aws_security_group.web_server.id
  cidr_blocks       = ["0.0.0.0/0"]
}

resource "aws_security_group_rule" "web_server_allow_ssh_inbound" {
  type              = "ingress"
  from_port         = 22
  to_port           = 22
  protocol          = "tcp"
  security_group_id = aws_security_group.web_server.id

  # Should be more restrictive in production:
  cidr_blocks = ["0.0.0.0/0"]
}

resource "aws_security_group_rule" "web_server_allow_all_outbound" {
  type              = "egress"
  from_port         = 0
  to_port           = 0
  protocol          = "-1"
  security_group_id = aws_security_group.web_server.id
  cidr_blocks       = ["0.0.0.0/0"]
}

EOF

In [13]:
cat secgroup.tf


resource "aws_security_group" "web_server" {
  name_prefix = "tfi-lab9"
}

resource "aws_security_group_rule" "web_server_allow_http_inbound" {
  type              = "ingress"
  
  # CHANGED to web_server_http_port:
  from_port         = var.web_server_http_port
  to_port           = var.web_server_http_port
  
  protocol          = "tcp"
  security_group_id = aws_security_group.web_server.id
  cidr_blocks       = ["0.0.0.0/0"]
}

resource "aws_security_group_rule" "web_server_allow_ssh_inbound" {
  type              = "ingress"
  from_port         = 22
  to_port           = 22
  protocol          = "tcp"
  security_group_id = aws_security_group.web_server.id

  # Should be more restrictive in production:
  cidr_blocks = ["0.0.0.0/0"]
}

resource "aws_security_group_rule" "web_server_allow_all_outbound" {
  type              = "egress"
  from_port         = 0
  to_port           = 0
  protocol          = "-1"
  security_group_id = aws_security_group.web_server.id
  cidr_blocks       =

We must also create a new *alb* security group allowing inbound traffic on the *alb_http_port* variable and all outbound traffic.

Add the security group and associated rules into main.tf.

* resource "aws_security_group" "alb"

* resource "aws_security_group_rule" "alb_allow_http_inbound"

* resource "aws_security_group_rule" "allow_all_outbound"

In [14]:
#EXCLUDE

cat > secgroup_alb.tf <<EOF

resource "aws_security_group" "alb" {
  name_prefix = "tfi-lab9-alb"
}

resource "aws_security_group_rule" "alb_allow_http_inbound" {
  type              = "ingress"
  
  # CHANGED to web_server_http_port:
  from_port         = var.alb_http_port
  to_port           = var.alb_http_port
  
  protocol          = "tcp"
  security_group_id = aws_security_group.alb.id
  cidr_blocks       = ["0.0.0.0/0"]
}

resource "aws_security_group_rule" "allow_all_outbound" {
  type              = "egress"
  from_port         = 0
  to_port           = 0
  protocol          = "-1"
  security_group_id = aws_security_group.alb.id
  cidr_blocks       = ["0.0.0.0/0"]
}
EOF


In [15]:
cat secgroup_alb.tf


resource "aws_security_group" "alb" {
  name_prefix = "tfi-lab9-alb"
}

resource "aws_security_group_rule" "alb_allow_http_inbound" {
  type              = "ingress"
  
  # CHANGED to web_server_http_port:
  from_port         = var.alb_http_port
  to_port           = var.alb_http_port
  
  protocol          = "tcp"
  security_group_id = aws_security_group.alb.id
  cidr_blocks       = ["0.0.0.0/0"]
}

resource "aws_security_group_rule" "allow_all_outbound" {
  type              = "egress"
  from_port         = 0
  to_port           = 0
  protocol          = "-1"
  security_group_id = aws_security_group.alb.id
  cidr_blocks       = ["0.0.0.0/0"]
}


### 2. Create a load balancer for the web servers.

<img src="ALB.PNG" />

It is also necessary to create the ALB resources.

* resource "aws_alb" "web_servers" - https://registry.terraform.io/modules/terraform-aws-modules/alb/aws/latest
   
* resource "aws_alb_listener" "http" - https://registry.terraform.io/providers/hashicorp/aws/latest/docs/resources/lb_listener

* resource "aws_alb_listener_rule" "send_all_to_web_servers" - https://registry.terraform.io/providers/hashicorp/aws/latest/docs/resources/lb_listener_rule

* resource "aws_alb_target_group" "web_servers" - https://registry.terraform.io/providers/hashicorp/aws/latest/docs/resources/lb_listener



Create the following alb.tf:

In [16]:
#EXCLUDE

cat > alb.tf <<EOF

resource "aws_alb" "web_servers" {
  name            = var.name
  security_groups = [aws_security_group.alb.id]
  subnets         = data.aws_subnets.default.ids

  lifecycle {
    create_before_destroy = true
  }
}

resource "aws_alb_listener" "http" {
  load_balancer_arn = aws_alb.web_servers.arn
  port              = var.alb_http_port
  protocol          = "HTTP"

  default_action {
    type             = "forward"
    target_group_arn = aws_alb_target_group.web_servers.arn
  }

  lifecycle {
    create_before_destroy = true
  }
}

resource "aws_alb_target_group" "web_servers" {
  name     = var.name
  port     = var.web_server_http_port
  protocol = "HTTP"
  vpc_id   = data.aws_vpc.default.id

  deregistration_delay = 10

  health_check {
    path                = "/"
    interval            = 15
    healthy_threshold   = 2
    unhealthy_threshold = 2
    timeout             = 5
  }

  lifecycle {
    create_before_destroy = true
  }
}

resource "aws_alb_listener_rule" "send_all_to_web_servers" {
  listener_arn = aws_alb_listener.http.arn
  priority     = 100

  action {
    type             = "forward"
    target_group_arn = aws_alb_target_group.web_servers.arn
  }

  condition {
    path_pattern {
      values = ["*"]
    }
  }
}

data "aws_vpc" "default" {
  default = true
}

# Attach this to the default VPC which we're using:

data "aws_subnets" "default" {
  filter {
    name   = "vpc-id"
    values = [ data.aws_vpc.default.id ]
  }
}

EOF

In [17]:
cat alb.tf


resource "aws_alb" "web_servers" {
  name            = var.name
  security_groups = [aws_security_group.alb.id]
  subnets         = data.aws_subnets.default.ids

  lifecycle {
    create_before_destroy = true
  }
}

resource "aws_alb_listener" "http" {
  load_balancer_arn = aws_alb.web_servers.arn
  port              = var.alb_http_port
  protocol          = "HTTP"

  default_action {
    type             = "forward"
    target_group_arn = aws_alb_target_group.web_servers.arn
  }

  lifecycle {
    create_before_destroy = true
  }
}

resource "aws_alb_target_group" "web_servers" {
  name     = var.name
  port     = var.web_server_http_port
  protocol = "HTTP"
  vpc_id   = data.aws_vpc.default.id

  deregistration_delay = 10

  health_check {
    path                = "/"
    interval            = 15
    healthy_threshold   = 2
    unhealthy_threshold = 2
    timeout             = 5
  }

  lifecycle {
    create_before_destroy = true
  }
}

resource "aws_alb_listener_rule" "send_all_to

### Add a new alb_http_port variable

Add the ```alb_http_port``` variable to the vars.tf file with a default value of 8080


In [18]:
#EXCLUDE

grep -q alb_http_port vars.tf || {

  cat >> vars.tf <<EOF

variable alb_http_port {
   default = 8080
}

EOF

}

cat > outputs.tf <<EOF

# Create an output to show the curl command to access the ALB:

output alb_curl_dns {
    value = [ for dns in aws_alb.web_servers.*.dns_name :
        "curl -L http://\${dns}:\${var.alb_http_port}"
    ]
}

EOF

In [19]:
cat outputs.tf


# Create an output to show the curl command to access the ALB:

output alb_curl_dns {
    value = [ for dns in aws_alb.web_servers.*.dns_name :
        "curl -L http://${dns}:${var.alb_http_port}"
    ]
}



In [20]:
TF_INIT


Initializing the backend...

Initializing provider plugins...
- Finding latest version of hashicorp/aws...
- Finding latest version of hashicorp/local...
- Finding latest version of hashicorp/tls...
- Installing hashicorp/aws v4.0.0...
- Installed hashicorp/aws v4.0.0 (signed by HashiCorp)
- Installing hashicorp/local v2.1.0...
- Installed hashicorp/local v2.1.0 (signed by HashiCorp)
- Installing hashicorp/tls v3.1.0...
- Installed hashicorp/tls v3.1.0 (signed by HashiCorp)

Terraform has created a lock file .terraform.lock.hcl to record the provider
selections it made above. Include this file in your version control repository
so that Terraform can guarantee to make the same selections by default when
you run "terraform init" in the future.

Terraform has been successfully initialized!

You may now begin working with Terraform. Try running "terraform plan" to see
any changes that are required for your infrastructure. All Terraform commands
should now work.

If you ever set or change 

In [21]:
TF_APPLY


Terraform used the selected providers to generate the following execution plan.
Resource actions are indicated with the following symbols:
  + create

Terraform will perform the following actions:

  # aws_alb.web_servers will be created
  + resource "aws_alb" "web_servers" {
      + arn                        = (known after apply)
      + arn_suffix                 = (known after apply)
      + desync_mitigation_mode     = "defensive"
      + dns_name                   = (known after apply)
      + drop_invalid_header_fields = false
      + enable_deletion_protection = false
      + enable_http2               = true
      + enable_waf_fail_open       = false
      + id                         = (known after apply)
      + idle_timeout               = 60
      + internal                   = (known after apply)
      + ip_address_type            = (known after apply)
      + load_balancer_type         = "application"
      + name                       = "alb-webserver"
      + security

      + name_prefix               = (known after apply)
      + protect_from_scale_in     = false
      + service_linked_role_arn   = (known after apply)
      + target_group_arns         = (known after apply)
      + vpc_zone_identifier       = [
          + "subnet-2c6ba676",
          + "subnet-b0e757d6",
        ]
      + wait_for_capacity_timeout = "10m"

      + tag {
          + key                 = "Name"
          + propagate_at_launch = true
          + value               = "alb-webserver"
        }
    }

  # aws_key_pair.generated_key will be created
  + resource "aws_key_pair" "generated_key" {
      + arn             = (known after apply)
      + fingerprint     = (known after apply)
      + id              = (known after apply)
      + key_name        = "asg-webservers-key"
      + key_name_prefix = (known after apply)
      + key_pair_id     = (known after apply)
      + public_key      = (known after apply)
      + tags_all        = (known after apply)
    }

  # aws

    }

  # aws_security_group_rule.web_server_allow_all_outbound will be created
  + resource "aws_security_group_rule" "web_server_allow_all_outbound" {
      + cidr_blocks              = [
          + "0.0.0.0/0",
        ]
      + from_port                = 0
      + id                       = (known after apply)
      + protocol                 = "-1"
      + security_group_id        = (known after apply)
      + self                     = false
      + source_security_group_id = (known after apply)
      + to_port                  = 0
      + type                     = "egress"
    }

  # aws_security_group_rule.web_server_allow_http_inbound will be created
  + resource "aws_security_group_rule" "web_server_allow_http_inbound" {
      + cidr_blocks              = [
          + "0.0.0.0/0",
        ]
      + from_port                = 8080
      + id                       = (known after apply)
      + protocol                 = "tcp"
      + security_group_id        = (known after 

aws_autoscaling_group.web_servers: Still creating... [30s elapsed]
aws_autoscaling_group.web_servers: Still creating... [40s elapsed]
aws_autoscaling_group.web_servers: Still creating... [50s elapsed]
aws_autoscaling_group.web_servers: Still creating... 0s elapsed]
aws_autoscaling_group.web_servers: Still creating... 10s elapsed]
aws_autoscaling_group.web_servers: Still creating... 20s elapsed]
aws_autoscaling_group.web_servers: Still creating... 30s elapsed]
aws_autoscaling_group.web_servers: Creation complete after 1m34s [id=terraform-20220215182316046700000003]

Apply complete! Resources: 16 added, 0 changed, 0 destroyed.

Outputs:

alb_curl_dns = [
  "curl -L http://alb-webserver-2101085059.us-west-1.elb.amazonaws.com:8080",
]
ssh_pem_key = <sensitive>
ssh_rsa_pub_key = <<EOT
ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAACAQCziWYKM5eEo5EH3VCdF6LYHT/eeamC0QKmmooh7yfJgVU3VLHCbf+7xo/O689bWoC8oIqGVqHReWRn68Q9nPczqw64v/NaMfuu6gGaIAzMJvUbtZQwessb4BKrDUK+EMSbk9UnG/lUp7ZDu3Ho5xmilgs/QcNh0cmS9d6KB/4aW

### 3. The configuration when visualized should look like

<img src="graph.svg" >

<div>
    <object data="graph.svg" type="image/svg+xml">
    </object>
</div>

### 4. Now apply the new config

At the end of the apply the url of the ALB should be output

### 5. Test the ALB is wotking

Now we test the ALB by performing a curl on the output URL, e.g.

```curl -L alb-webserver-687206123.us-west-1.elb.amazonaws.com:8080```

In [22]:
#EXCLUDE
#set -x
#set +x
__CURL_CMD="terraform output alb_curl_dns | sed 's/,//' | jq -r '.[]'"

__CURL_CMD=$( terraform output alb_curl_dns | sed 's/,//' | jq -r '.[]' )
#__CURL_CMD="terraform output alb_curl_dns"
#eval $__CURL_CMD

echo $__CURL_CMD
$__CURL_CMD
#terraform output alb_curl_dns

curl -L http://alb-webserver-2101085059.us-west-1.elb.amazonaws.com:8080
from ip-172-31-4-214: Your server is up !!


Verify that the reported hostname changes due to the Round Robin Load Balancing, e.g.

In [23]:
CURL_CMD=$( terraform output alb_curl_dns | sed 's/,//' | jq -r '.[]' )

$CURL_CMD

from ip-172-31-31-76: Your server is up !!


In [24]:

let LOOP=10

CURL_CMD=$( terraform output alb_curl_dns | sed 's/,//' | jq -r '.[]' )

while [ $LOOP -gt 0 ]; do
    $CURL_CMD 
    let LOOP=LOOP-1
done


from ip-172-31-25-169: Your server is up !!
from ip-172-31-4-214: Your server is up !!
from ip-172-31-31-76: Your server is up !!
from ip-172-31-25-169: Your server is up !!
from ip-172-31-4-214: Your server is up !!
from ip-172-31-25-169: Your server is up !!
from ip-172-31-31-76: Your server is up !!
from ip-172-31-4-214: Your server is up !!
from ip-172-31-25-169: Your server is up !!
from ip-172-31-4-214: Your server is up !!


: 1

### 6. Cleanup

In [25]:
TF_DESTROY

tls_private_key.mykey: Refreshing state... [id=de3a880ca2b93a143134fbbf6631390932ba7380]
local_file.pem_key: Refreshing state... [id=9fb89da9e2e22fa5d8f06093e881dacfb2d10f71]
aws_security_group.web_server: Refreshing state... [id=sg-02496ec16f417eadd]
aws_key_pair.generated_key: Refreshing state... [id=asg-webservers-key]
aws_security_group.alb: Refreshing state... [id=sg-0edfc6c213af5564c]
aws_security_group_rule.alb_allow_http_inbound: Refreshing state... [id=sgrule-9677397]
aws_security_group_rule.allow_all_outbound: Refreshing state... [id=sgrule-1034742298]
aws_security_group_rule.web_server_allow_ssh_inbound: Refreshing state... [id=sgrule-1427723981]
aws_security_group_rule.web_server_allow_all_outbound: Refreshing state... [id=sgrule-486124735]
aws_security_group_rule.web_server_allow_http_inbound: Refreshing state... [id=sgrule-344744221]
aws_launch_configuration.web_servers: Refreshing state... [id=terraform-20220215182316046700000003]
aws_alb_target_group.web_servers: Refres

          - type             = "forward" -> null
        }

      - condition {

          - path_pattern {
              - values = [
                  - "*",
                ] -> null
            }
        }
    }

  # aws_alb_target_group.web_servers will be destroyed
  - resource "aws_alb_target_group" "web_servers" {
      - arn                                = "arn:aws:elasticloadbalancing:us-west-1:604682923221:targetgroup/alb-webserver/cc79996f84704451" -> null
      - arn_suffix                         = "targetgroup/alb-webserver/cc79996f84704451" -> null
      - connection_termination             = false -> null
      - deregistration_delay               = "10" -> null
      - id                                 = "arn:aws:elasticloadbalancing:us-west-1:604682923221:targetgroup/alb-webserver/cc79996f84704451" -> null
      - lambda_multi_value_headers_enabled = false -> null
      - load_balancing_algorithm_type      = "round_robin" -> null
      - name                       

      - fingerprint = "c5:49:b3:11:e8:73:85:cf:32:02:1b:f1:80:34:47:fa" -> null
      - id          = "asg-webservers-key" -> null
      - key_name    = "asg-webservers-key" -> null
      - key_pair_id = "key-0f6db23c0b8850db2" -> null
      - public_key  = "ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAACAQCziWYKM5eEo5EH3VCdF6LYHT/eeamC0QKmmooh7yfJgVU3VLHCbf+7xo/O689bWoC8oIqGVqHReWRn68Q9nPczqw64v/NaMfuu6gGaIAzMJvUbtZQwessb4BKrDUK+EMSbk9UnG/lUp7ZDu3Ho5xmilgs/QcNh0cmS9d6KB/4aWjwHYLmnBcR+CbTZkBQz8io7lmbqqPDragQV4h6XxqzJIF6CRRRpsRRhypdeGmhZ3BDacEu0UkSj/73Ucyotgu+DqArJncX9n2+cDmNZzMWqfshUtCDQkhSfBqlxsSJwdCBFFuKZKHa7eZpHUhIFbPUREi9tGKPHORYSsOdgH5tNsAER3SLJ8WGoAMenVa3lr5KVnXalB0HMdjILsdPRZhtmK6mqkf4lMs+3FGezuOpBt3H2w3+RYeyIU/XH2PLPCpRGxDc1ISVdzvntlnznrmxFv70zGTe3phtevEpI+trAdvN+ha8mkVxkVnZWHz6OJh64Kyu2UYmrneXBTCXueh1jNVxBKWIab71EAdtn85NRv4yb0jGIkkbtepA6CJ++/k/BHTYrgUo4aMHODPY7CorWapxrKX8qIQAyZEssfr6woKy5d+vX4NDUC1KBjxkeaipoC9lJ0OBbIne08FvEkF2mbTyc3B/VYjA+YN84J08qNAjo7iUuvJd6KQpr+l39qQ==" -> null
      -

              - ipv6_cidr_blocks = []
              - prefix_list_ids  = []
              - protocol         = "tcp"
              - security_groups  = []
              - self             = false
              - to_port          = 8080
            },
        ] -> null
      - name                   = "tfi-lab920220215182309140600000002" -> null
      - name_prefix            = "tfi-lab9" -> null
      - owner_id               = "604682923221" -> null
      - revoke_rules_on_delete = false -> null
      - tags                   = {} -> null
      - tags_all               = {} -> null
      - vpc_id                 = "vpc-af4583c9" -> null
    }

  # aws_security_group_rule.alb_allow_http_inbound will be destroyed
  - resource "aws_security_group_rule" "alb_allow_http_inbound" {
      - cidr_blocks       = [
          - "0.0.0.0/0",
        ] -> null
      - from_port         = 8080 -> null
      - id                = "sgrule-9677397" -> null
      - ipv6_cidr_blocks  = [] -> null
      

      - id                         = "de3a880ca2b93a143134fbbf6631390932ba7380" -> null
      - private_key_pem            = (sensitive value)
      - public_key_fingerprint_md5 = "df:f5:d4:48:42:30:18:c5:9c:de:53:a4:d8:d2:1d:b3" -> null
      - public_key_openssh         = <<-EOT
            ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAACAQCziWYKM5eEo5EH3VCdF6LYHT/eeamC0QKmmooh7yfJgVU3VLHCbf+7xo/O689bWoC8oIqGVqHReWRn68Q9nPczqw64v/NaMfuu6gGaIAzMJvUbtZQwessb4BKrDUK+EMSbk9UnG/lUp7ZDu3Ho5xmilgs/QcNh0cmS9d6KB/4aWjwHYLmnBcR+CbTZkBQz8io7lmbqqPDragQV4h6XxqzJIF6CRRRpsRRhypdeGmhZ3BDacEu0UkSj/73Ucyotgu+DqArJncX9n2+cDmNZzMWqfshUtCDQkhSfBqlxsSJwdCBFFuKZKHa7eZpHUhIFbPUREi9tGKPHORYSsOdgH5tNsAER3SLJ8WGoAMenVa3lr5KVnXalB0HMdjILsdPRZhtmK6mqkf4lMs+3FGezuOpBt3H2w3+RYeyIU/XH2PLPCpRGxDc1ISVdzvntlnznrmxFv70zGTe3phtevEpI+trAdvN+ha8mkVxkVnZWHz6OJh64Kyu2UYmrneXBTCXueh1jNVxBKWIab71EAdtn85NRv4yb0jGIkkbtepA6CJ++/k/BHTYrgUo4aMHODPY7CorWapxrKX8qIQAyZEssfr6woKy5d+vX4NDUC1KBjxkeaipoC9lJ0OBbIne08FvEkF2mbTyc3B/VYjA+YN84J08qNAjo7i


Destroy complete! Resources: 16 destroyed.


To destroy the formerly created AWS vpc, and all subnets.

<hr/>
<!-- ![](../../../static/images/LOGO_v2_CROPPED.jpg) --> <img src="../../../static/images/LOGO_v2_CROPPED.jpg" width="200" />

# --EXCLUDE--SECTION--

## **NOTE:** Save notebook first

<div id="END" /> <a href="#TOP" > TOP OF PAGE </a>  <!-- must appear after EXCLUDE pragma !! -->

In [27]:
NB_DIR=~/nb/2022-Terraform/content/tf-labs/9.ScalingAndLoadBalancingWebservers/
cd $NB_DIR

. ~/scripts/nbtool.rc
pwd
EXCL_FN_FILTER_NOTEBOOK $NB

/home/student/nb/2022-Terraform/content/tf-labs/9.ScalingAndLoadBalancingWebservers

==== Filtering current notebook [README.ipynb] ====
-- . ~/scripts/nbtool.rc

-- /usr/bin/python3 ~/scripts/nbtool.py -f README.ipynb
---- no VARS_SEEN ----
i/p nb has 0 variables
README.ipynb:
	#cells=43
cells to include[#27]=[[4, 5, 6, 7, 9, 10, 11, 14, 15, 16, 18, 20, 21, 23, 24, 26, 27, 29, 30, 31, 32, 34, 35, 36, 37, 38, 39]]
cells to include[#27]=[[39, 38, 37, 36, 35, 34, 32, 31, 30, 29, 27, 26, 24, 23, 21, 20, 18, 16, 15, 14, 11, 10, 9, 7, 6, 5, 4]]
included markdown cells=[4, 7, 11, 16, 18, 21, 24, 27, 32, 34, 37, 39]
included code     cells=[5, 6, 9, 10, 14, 15, 20, 23, 26, 29, 30, 31, 35, 36, 38]
excluded          cells=[0, 1, 2, 3, 8, 12, 13, 17, 19, 22, 25, 28, 33, 40, 41]
---- no VARS_SEEN ----
o/p nb has 0 variables
README.ipynb.filtered.ipynb:
	#cells=27
-rw-r--r-- 1 student docker 76770 Feb 15 18:31 README.ipynb.filtered.ipynb

6 VAR defs/usages in i/p file: README.ipynb

Good: No VAR d